Customer cohorts  are tracked by start month and active month. Start month is the month during which a cohort of customers became active.  So, for every month, there is a  cohort of new customers.

All cohorts are tracked during the entire year. Given any cohort, there is expected to be a drop-off of customers that belong to that cohort.

The customer count of a cohort in the second and subsequent month of a cohort's existence is referred  as retention counts, while the number recorded in the first month is the initial count.

Churn rate gauges the fall-off in customer retention across all cohorts. Given a 12 month calendar period, where the tracking activity is assumed to start in January, the 1st churn rate is the churn rate for February. That number is calculated by summing the customer counts across all cohorts in the
2nd month of their existence, divided by the sum of customer counts of cohorts in the 1st month of their existence, minus 1.

Similarly, the 2nd churn rate id calculated thusly:sum of customer counts for all cohorts in their 3rd month of existence/sum of customer counts for all cohorts in their 2nd month of existence -1;

The 3rd and subsequent churn rates are calculated similarly.

Generalizing,  the nth churn rate is: sum of customer counts for all cohorts in their n+1st month of existence/sum of customer counts for all cohorts in their nth month of existence -1;

The input data needed are rows of [sorted] data with the following columns:
start-month, active-month, customer-count

A row with start-month and active-month are the same signals the start of a cohort. Once that is seen,
the start-month identifies a cohort; the series of counts associated with the subsequent active months (for that start-month) represents the retention counts for that cohort.

Other processing considerations:
1. The cohort associated with the final period (i.e last  active month) is not considered in churn rate
    calculations as there is no retention count data available for that cohort
2. The life span of cohorts varies from the earliest formed cohort to the most recently formed one.
   Therefore when processing for the nth month, a cohort may not exist and so it has to be by-passed   
   for aggregation.

Customer Cohort Monthly churn rate calculator : Calculates monthly churn rate and forecast of customer count

Input to this notebook operation : A csv file that captures an SQL Query resultset of normalized customer cohort data. For each period that data is the cohort start month,the active month and the associated recorded customer count

In [13]:
customActualsInputFile = "../csv/customer-actuals.csv" # input to start notebook operations

In [14]:
customerActualsMonthlyChurnRate = []    # output of the notebook operation; persisted into Pod#UnitCustomerLTV/VAR#monthly-churn-rate

Output of the notebook operation: 
   customerActualsMonthlyChurnRates array
   The monthly churn rate (on a weighted basis across all cohorts) from the beginning 
   month to the next to last month. Persisted into Pod#UnitCustomerLTV as VAR monthly-churn-rate 


In [15]:
normalizedCustomerActualsJsonData = [];

In [16]:
cusActualsPodName= "CustomerActuals"; # name of associated pod
unitCusLTVPodName= "UnitCustomerLTV";  # name of related pod

import sys; sys.path.insert(0, '../pythonLib')
from customerCohortOperations import prepareDataForPopulatingActualsOrForecastPod, updatePodObj
from LexoOperations import getBean, saveBean, getPod, savePod

In [17]:
# convert cohort csv data into a json (python Dict) object 
import csv
import json

def csv_to_json(csvFilePath):
    
    # read csv file and populate python object normalizedCustomerCohortJsonData
    with open(csvFilePath, encoding='utf-8') as csvf:
        # load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf)

        # convert each csv row into python dict
        for row in csvReader:
            # add this python dict to json array
            normalizedCustomerActualsJsonData.append(row)

csvFilePath =customActualsInputFile
csv_to_json(csvFilePath)

#print(normalizedCustomerActualsJsonData)



In [18]:
from IPython.display import JSON
import requests
import pandas as pd
import  json
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [19]:
def numberOfMonths( dateOfFirstPeriod, dateOfLastPeriod):
    date_format = '%Y-%m-%d';
    end_date = datetime.strptime(dateOfLastPeriod,date_format)
    start_date = datetime.strptime(dateOfFirstPeriod,date_format)

    num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) +1;
    return num_months;

def addMonthsToGivenDate( givenDate, n):
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(givenDate, date_format)
    futureDate = dtObj + relativedelta(months=n);
    futureDateStr = futureDate.date().strftime(date_format);
    return futureDateStr


In [20]:
def getCusCountFromDataFrameGivenActiveMonthAndStartMonth(normalizedDF, activeMonth, startMonth,):
    df = normalizedDF;
    cusCount = float("NAN");
    selectedRow = df.loc[(df['active-month'] == activeMonth) & (df['start-month'] == startMonth)]  # returns a dataframe with that one row
    if selectedRow.empty :
        pass
    else:
        cusCount =  selectedRow.iloc[0,2]  #1st index is row selector, 2nd index is col selector (cus-count is at index 2
    if (isinstance(cusCount,int)):
        return (int(cusCount))
    else:
        return float(cusCount)

In [21]:
def calcMonthlyChurnRate(  normalizedDF,  activeMonths, startMonths):

    monthlyChurnRate =[];
    firstPeriodDate = activeMonths[0];
    lastPeriodDate = activeMonths[-1];
    numOfMonthsInEvalPeriod = numberOfMonths(firstPeriodDate, lastPeriodDate);
    
    periodIndex = 0;
    for activeMonth in activeMonths:
        sumOfRetainedCustomersForActiveMonth= 0;
        periodIndex = periodIndex+1;
        #churnrate not calculated for last month in period as it  requires data for following month
        if periodIndex >= numOfMonthsInEvalPeriod:
            pass
        else:
            #print('activeMonth:', activeMonth)
            baselineCustomersForActiveMonth = 0;
            for startMonth in startMonths:
                cusStartDate = startMonth;
                trackingDate = addMonthsToGivenDate(cusStartDate, periodIndex);
                if trackingDate > lastPeriodDate :
                    continue
                else:
                    cusCount = getCusCountFromDataFrameGivenActiveMonthAndStartMonth(normalizedDF,trackingDate, cusStartDate)
                   
                    if  pd.isna(cusCount):
                        continue
                    else:
                    
                        sumOfRetainedCustomersForActiveMonth = sumOfRetainedCustomersForActiveMonth + cusCount;
                        #print('sumOfRetainedCustomersForActiveMonth:', sumOfRetainedCustomersForActiveMonth);
                    #end
                    # sum of all baseline customer count in every cohort for every period until the next to last month
                    baselineTrackingDate = addMonthsToGivenDate(cusStartDate, periodIndex - 1);
                    baselineCusCount = getCusCountFromDataFrameGivenActiveMonthAndStartMonth(normalizedDF,baselineTrackingDate, cusStartDate)
                    #print('activeMonth:', baselineTrackingDate, 'startMonth:',cusStartDate, 'baselineCusCount:', baselineCusCount);
                    if  pd.isna(baselineCusCount):
                        pass
                    else:
                        baselineCustomersForActiveMonth = baselineCustomersForActiveMonth + baselineCusCount;
                        #print('baselineCustomersForActiveMonth:', baselineCustomersForActiveMonth);
            if baselineCustomersForActiveMonth > 0:
                churnRate = sumOfRetainedCustomersForActiveMonth/baselineCustomersForActiveMonth - 1;
                #print('activeMonth:', activeMonth, 'churnRate:', churnRate);
                monthlyChurnRate.append(churnRate);
        #end of else
    return monthlyChurnRate;


In [22]:
def extractStartMonthsAndActiveMonths(normalizedActualCustomerCountJsonData):
    startMonths=[];
    activeMonths=[];
    for customerActivityRecord in normalizedActualCustomerCountJsonData:
        startMonth = customerActivityRecord["start-month"]
        activeMonth = customerActivityRecord["active-month"]
        cusCount=customerActivityRecord["cus-count"]
        if startMonth not in startMonths:
            startMonths.append(startMonth)
        if activeMonth not in activeMonths:
            activeMonths.append(activeMonth)
                
    retVal = { "startMonths":startMonths, "activeMonths":activeMonths}     
    return retVal;


In [23]:
#load the customer actuals json data into a data frame
normalizedCustomerActualsDF = pd.DataFrame(normalizedCustomerActualsJsonData)
#print(normalizedCustomerActualsDF)

In [24]:
#pivoted customer cohort DataFrame
pivotedCustomerActualsDF = pd.pivot_table(normalizedCustomerActualsDF, values='cus-count', 
                                         index=['active-month'],columns=['start-month'])

print(pivotedCustomerActualsDF)

start-month   2020-01-01  2020-02-01  2020-03-01  2020-04-01  2020-05-01  \
active-month                                                               
2020-01-01          50.0         NaN         NaN         NaN         NaN   
2020-02-01          35.0        55.0         NaN         NaN         NaN   
2020-03-01          30.1        38.5        60.0         NaN         NaN   
2020-04-01          25.0        33.1        42.0        65.0         NaN   
2020-05-01          25.0        27.5        36.1        45.5        70.0   
2020-06-01          25.0        27.5        30.0        39.1        49.0   
2020-07-01          25.0        27.5        30.0        32.5        42.1   
2020-08-01          25.0        27.5        30.0        32.5        35.0   
2020-09-01          25.0        27.5        30.0        32.5        35.0   
2020-10-01          25.0        27.5        30.0        32.5        35.0   
2020-11-01          25.0        27.5        30.0        32.5        35.0   
2020-12-01  

In [25]:
retVal = extractStartMonthsAndActiveMonths(normalizedCustomerActualsJsonData)
startMonths =retVal["startMonths"]
activeMonths = retVal["activeMonths"]


In [26]:
customerActualsMonthlyChurnRate = calcMonthlyChurnRate(normalizedCustomerActualsDF,startMonths,activeMonths)
#print(customerActualsMonthlyChurnRate)

In [27]:

# set number of decimal places and make it string; 
# it turns out that floats are not retaining 2 decimals if value is 0; better consistency if string

mcr = []
for item in customerActualsMonthlyChurnRate:
    #set number of decimal points to 2 and make it a float from string
    if (isinstance(item,str)) :
        item = '%.2f'%float(item) #use the string format operator %; 
        mcr.append(item) 
    elif (isinstance(item,float)):
        item = '%.2f'%item #will trim to 2 decimals and make it str
        mcr.append(item)
    else:   
        mcr.append(item)
customerActualsMonthlyChurnRate = mcr
print(customerActualsMonthlyChurnRate)

['-0.30', '-0.14', '-0.17', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']


In [28]:
%store customerActualsMonthlyChurnRate

Stored 'customerActualsMonthlyChurnRate' (list)


At this point the churn rates have been calculated from the normalized cohort data input;
The CustomerActuals Pod needs to be updated ( it represents a pivoted view of the normalized structure). It is assumed that the Pod has already been set up( with the appropriate column and row headers). Data to be populated is prepared ; the pod is fetched, updated with the prepared data and then saved

In [107]:
podDataForCusActuals = prepareDataForPopulatingActualsOrForecastPod(normalizedCustomerActualsDF);

#For debugging, uncomment
#for item in podDataForCusActuals:
#    print(item)

In [108]:
cusActualsPod = getPod(cusActualsPodName)
#JSON(cusActualsPod)

In [109]:
#update CustomerActuals data 
updatePodObj(cusActualsPod, podDataForCusActuals) # uncomment to execute

In [110]:
savedPod= savePod(cusActualsPod)  #uncomment to execute

In [111]:

#persist var value for monthly churn rate; source of truth is pod UnitCustomerLTV
varsDelta = {"monthly-churn-rate":customerActualsMonthlyChurnRate}

#fetch the associated pod and load the vars object
pod = getPod(unitCusLTVPodName);
podVars = pod['vars'];
#update the loaded pod Vars with vars Delta produced from this notebook operation
podVars.update(varsDelta);
#prepare payload for pod update
podPayload =  {"_id":pod['_id'],"vars":podVars}
savedPod=savePod(podPayload)

JSON(savedPod)

<IPython.core.display.JSON object>